The notebooks combines the data. It excludes anyone who has played fewer than 675 minutes and who is no longer at a club in the top-5 league. To be in the initial dataset the player also have played in the 2020-21 season in the big-5 leagues.

In [1]:
import pandas as pd
import numpy as np
import os
import fuzzymatcher
from scrape_utils import get_fbref_player_dob
import time

# Loading fbref data

In [2]:
df = pd.read_parquet(os.path.join('data', 'fbref_combined.parquet'))
df['player_name'] = df.player.str.replace("'", "")

OSError: NotImplemented: Support for codec 'snappy' not built

# Filter players with 7.5 90s or over (675 minutes)

In [3]:
df = df[df.playing_time_min >= 675].copy()

# Loading transfermarkt data

In [4]:
df_tm = pd.read_parquet(os.path.join('data', 'players_transfermarket.parquet'))
df_tm['born'] = df_tm.dob.dt.year
df_tm['player_name'] = df_tm.player.str.replace("'", "")
#df_tm_2020 = df_tm[df_tm.year == '2020'].copy()
#df_tm = df_tm[df_tm.year == '2019'].copy()

# Deduplicate transfermarkt data

In [5]:
df_tm.sort_values('joined', inplace=True)
df_tm.drop_duplicates('player_link', inplace=True, keep='last')

# Adding on fbref squad name to transfermarkt data

In [6]:
# replacing some team names
team_replace = {'Borussia Mönchengladbach': "M'gladbach",
                'Stade Rennais FC': 'Rennes',
                'AC Milan': 'Milan',
                'Inter Milan': 'Inter',
                'West Bromwich Albion': 'West Brom',
                'West Ham United': 'West Ham',
                'Wolverhampton Wanderers': 'Wolves'}
df_tm.team_name.replace(team_replace, inplace=True)
# matching team names
df_team1 = pd.DataFrame(df.squad.drop_duplicates().reset_index(drop=True))
df_team2 = pd.DataFrame(df_tm.team_name.drop_duplicates().reset_index(drop=True))
df_team_match = fuzzymatcher.fuzzy_left_join(df_team1,
                                             df_team2,
                                             left_on=['squad'],
                                             right_on=['team_name'])
df_team_match = df_team_match[['team_name', 'squad']].copy()
# add on fbref squad name to transfermarkt data
df_tm = df_tm.merge(df_team_match, on='team_name')

# Matching fbref top-5 table to transfermarkt data

In [7]:
# merging based on player name and year of birth
df_player_match = fuzzymatcher.fuzzy_left_join(df[['player_name', 'player_link', 'pos', 'squad', 'born']],
                                               df_tm,
                                               left_on=['player_name', 'born'],
                                               right_on=['player_name', 'born'],
                                               left_id_col='player_link',
                                               right_id_col='player_link')
# rules for splitting matches and non-matches
mask_match1 = ((df_player_match.squad_left == df_player_match.squad_right) & 
               ((abs(df_player_match.born_left - df_player_match.born_right) <= 4) | (df_player_match.born_left.isnull())) &
               (df_player_match.best_match_score > -0.04))
mask_match2 = ((df_player_match.squad_left != df_player_match.squad_right) & 
               (df_player_match.born_left == df_player_match.born_right) &
               (df_player_match.best_match_score >= 0.1))
matched = df_player_match[mask_match1 | mask_match2].copy()
matched.sort_values('best_match_score', inplace=True)
matched.drop_duplicates('__id_right', keep='last', inplace=True)
not_matched = df[~df.player_link.isin(matched.__id_left)]
print('Number matched:', len(matched), 'Number unmatched:', len(not_matched))

Number matched: 2025 Number unmatched: 81


# For unmatched data matching get the more detailed biographical data from fbref (player name and dob)

In [8]:
#links = not_matched.player_link.tolist()
#links2 = [f'https://fbref.com{s}' for s in links]

In [9]:
#names_list = []
#dob_list = []
#squad_list = []
#for url in links2:
#    name, dob, squad = get_fbref_player_dob(url)
#    names_list.append(name)
#    dob_list.append(dob)
#    squad_list.append(squad)
#    time.sleep(30)
#df_name_dob = pd.DataFrame({'player_link': links, 'player': names_list, 'dob': dob_list, 'squad': squad_list})
#df_name_dob.to_parquet(os.path.join('data', 'fbref_name_dob.parquet'))

In [10]:
df_name_dob = pd.read_parquet(os.path.join('data', 'fbref_name_dob.parquet'))

# Merge on the more detailed biographical data

In [11]:
# merging based on player name and date of birth
df_player_match2 = fuzzymatcher.fuzzy_left_join(df_name_dob,
                                                df_tm,
                                                left_on=['player', 'dob'],
                                                right_on=['player_name', 'dob'],
                                                left_id_col='player_link',
                                                right_id_col='player_link')
# rules for splitting matches and non-matches
mask_match = (df_player_match2['best_match_score'] > 0) & (df_player_match2['dob_left'] == df_player_match2['dob_right'])
matched2 = df_player_match2[mask_match].copy()
not_matched2 = df_player_match2[~mask_match].copy()
print('Number matched:', len(matched2), 'Number unmatched:', len(not_matched2))

Number matched: 12 Number unmatched: 69


# Get un-matched still in top-5 leagues

In [12]:
not_matched2_big5 = not_matched2[not_matched2.squad_left.isin(df_team_match.squad)]
not_matched2_outside_big5 = not_matched2[~not_matched2.squad_left.isin(df_team_match.squad)]
print('Number unmatched (outside the big 5 leagues): ', len(not_matched2_outside_big5))
print('Number unmatched (playing in big 5 leagues): ', len(not_matched2_big5))

Number unmatched (outside the big 5 leagues):  60
Number unmatched (playing in big 5 leagues):  9


# I have manually found the links for 7 of the remaining 9 unmatched players in the big 5 leagues through google search/ str lookups (the other 2 have left the league)

In [13]:
fb_links = ['/en/players/80dbbd10/Simeon-Nwankwo',
            '/en/players/faed84db/Uros-Racic',
            '/en/players/eac044ad/Moanes-Dabour',
            '/en/players/3423f250/Raphael-Dias-Belloli',
            #'/en/players/d14229f3/Adrien-Hunou',
            '/en/players/c6e8cf1f/Sasa-Lukic',
            #'/en/players/2054d037/MBaye-Niang',
            '/en/players/3ae14ed1/Trezeguet',
            '/en/players/79443529/Dusan-Vlahovic']
tm_links = ['/simy/profil/spieler/194549',
            '/uros-racic/profil/spieler/417575',
            '/munas-dabbur/profil/spieler/145866',
            '/raphinha/profil/spieler/411295',
            #'/adrien-hunou/profil/spieler/203507',
            '/sasa-lukic/profil/spieler/245056',
            #'/mbaye-niang/profil/spieler/157501',
            '/trezeguet/profil/spieler/234189',
            '/dusan-vlahovic/profil/spieler/357498']
matched3 = pd.DataFrame({'player_link_left': fb_links, 'player_link_right': tm_links})

# Combine matches into a single table of links

In [14]:
all_matched = pd.concat([matched3,
                         matched2[['player_link_left', 'player_link_right']],
                         matched[['player_link_left', 'player_link_right']]])
all_matched.rename({'player_link_left': 'player_link_fbref', 'player_link_right': 'player_link_tm'},
                   axis='columns', inplace=True)
print('Check no duplicated fbref links:', all_matched.duplicated('player_link_fbref', keep=False).sum())
print('Check no duplicated transfermarkt links:', all_matched.duplicated('player_link_tm', keep=False).sum())
print('Check there are the same players in each dataset (i.e. the symmetric difference is empty):',
      set(all_matched.player_link_fbref).symmetric_difference(set(df.player_link)) 
      - set(not_matched2_outside_big5.player_link_left)) # remove players who are not playing in the big-5 leagues currently
print('Number of matched:', len(all_matched))

Check no duplicated fbref links: 0
Check no duplicated transfermarkt links: 0
Check there are the same players in each dataset (i.e. the symmetric difference is empty): {'/en/players/d14229f3/Adrien-Hunou', '/en/players/2054d037/MBaye-Niang'}
Number of matched: 2044


# Filter matched datasets

In [17]:
df_tm = df_tm[df_tm.player_link.isin(all_matched.player_link_tm)].copy()
df = df[df.player_link.isin(all_matched.player_link_fbref)].copy()
print(len(df_tm), len(df))

2044 2044


# Calculate age at 1st December 2020

In [18]:
df_tm['age_at_2021_04_01'] =  np.round((pd.to_datetime('2021-04-01') - df_tm.dob).dt.days / 365.25, 1)

# Format and save dataframe

In [22]:
df_tm.drop(['team_name', 'player_name', 'transfer_details', 'born', 'signed_from_link'], axis='columns', inplace=True)
df_tm['year'] = pd.to_numeric(df_tm.year, errors='coerce')
df.rename({'year': 'market_value_year'}, axis='columns', inplace=True)
cols = df_tm.columns
df_tm.columns = [c+'_tm' for c in cols]
df_tm = df_tm.merge(all_matched, how='inner', validate='1:1', on='player_link_tm')
df.rename({'player_link': 'player_link_fbref'}, axis='columns', inplace=True)
df.drop('player_name', axis='columns', inplace=True)
df = df_tm.merge(df, how='right', on='player_link_fbref', validate='1:1')
df.to_parquet(os.path.join('data', 'fbref_tm_combined.parquet'))

In [23]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2044 entries, 0 to 2043
Data columns (total 173 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   number_tm                      2044 non-null   float64       
 1   player_tm                      2044 non-null   object        
 2   position_tm                    2044 non-null   object        
 3   height_tm                      2044 non-null   float64       
 4   foot_tm                        2044 non-null   object        
 5   joined_tm                      2043 non-null   datetime64[ns]
 6   contract_expires_tm            2023 non-null   datetime64[ns]
 7   market_value_euro_millions_tm  2044 non-null   float64       
 8   league_tm                      2044 non-null   object        
 9   player_link_tm                 2044 non-null   object        
 10  year_tm                        2044 non-null   int64         
 11  dob_tm          